# Draft analysis 

---

Group name: Lena Breitberg, Ji Huixiao, Paraskevas Papadopoulos

---


## Introduction

*This section includes an introduction to the project motivation, data, and research question. Include a data dictionary* 

## Setup

In [1]:
import numpy as np
import pandas as pd

from scipy import stats
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns  

import altair as alt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Data

## Import data

In [2]:
# import data
ROOT = "https://raw.githubusercontent.com/hdm-statistik-lpj/project/main/data/external/"
DATA = "immobilienNuernberg.csv"
df = pd.read_csv(ROOT + DATA)

### Data structure

In [3]:
# show the first rows
pd.set_option('display.max_columns', None)
df.head()

,General.EstateTypeKey,General.ReferenceNumber,General.DistributionTypeKey,General.ConstructionYear,General.LivingSpace,Tealium.object_locationid,Tealium.object_address_is_visible,Tealium.object_zip,Tealium.object_currency,Tealium.object_features,Tealium.object_objektart_sub,Tealium.expose_type,Tealium.broker_guid,Tealium.object_count_photos,LocalRatings.scores.local_amenities,LocalRatings.scores.mobility,MetaBadges,Price.AdditionalInformation.Commission.CommissionType,EstateAddress.City,EstateAddress.ZipCode,EstateAddress.Street,Offerer.addressGuid,Offerer.globalUserId,Offerer.contactData.companyName,Project.Address.PostCode,Project.Address.Street,Project.Address.HouseNumber,Latitude,Longitude,Stadtteil,Stadt,PRICE,ROOMS,PRICE_COMMONCHARGE,EnergyType,Class,Value,CATEGORY,FLOOR,USAGE,SUITABILITY,CONDITION,HOUSECONDITION,PARKINGSLOT,ENERGY,BUILDINGTYPE,HEATING,ELEVATOR,EQUIPMENTS
0,WOHNUNG,001/G1/E114,ZUM_KAUF,2023.0,247.07,493480,True,90482,EUR,"['Neubau', 'WANNE', 'gaestewc', 'Bad/WC getren...",wohnung,project,4821481,15.0,7.7,9.5,"[{'label': 'Provisionsfrei', 'highlight': True}]",COMMISSION_FREE,Nürnberg,90482,Ostendstraße 120,69C9B07CBE7147ED868C55D3368C8261,4821481,Instone Real Estate Development GmbH Niederlas...,90482.0,Ostendstraße,115,49.45760,11.13188,Mögeldorf,Nürnberg,2539000.0,7.0,NaN,NaN,NaN,NaN,NaN,11. Geschoss,NaN,NaN,Erstbezug,Neubau,Tiefgarage,NaN,NaN,Fußbodenheizung,Personenaufzug,NaN
1,WOHNUNG,NaN,ZUM_KAUF,1900.0,69.00,493430,False,90461,EUR,"['Dach ausgebaut', 'WANNE', 'gaestewc', 'Bad/W...",MAISONETTE,standard,2704438,23.0,6.5,9.7,[],COMMISSION_REQUIRED,Nürnberg,90461,NaN,96453B9B5899480AB506126D5E0E9140,2704438,Immobilien Büro Franken24 UG (haftungsbeschränkt),NaN,NaN,NaN,49.42749,11.07581,Gibitzenhof,Nürnberg,225000.0,2.0,280.0,NaN,NaN,NaN,Maisonette,4. Geschoss (Dachgeschoss),vermietet,WG-geeignet,renoviert / saniert,NaN,NaN,Gas,NaN,Etagenheizung,NaN,NaN
2,WOHNUNG,NaN,ZUM_KAUF,1972.0,69.46,493465,False,90473,EUR,"['WANNE', 'GEPFLEGT', 'FERN', 'LAMINAT', 'FLIE...",wohnung,standard,7111707,1.0,5.7,7.8,[],COMMISSION_DEFAULT,Nürnberg,90473,NaN,9C587ED116BD427E9DD13790553F0A5F,7111707,W &a; F Liegenschaften GmbH,NaN,NaN,NaN,49.40815,11.13376,Langwasser,Nürnberg,195000.0,3.0,283.0,NaN,NaN,NaN,NaN,8. Geschoss,vermietet,NaN,gepflegt,NaN,Tiefgarage,Fernwärme,NaN,NaN,NaN,NaN
3,WOHNUNG,X4_301,ZUM_KAUF,2023.0,82.81,493524,True,90429,EUR,"['Neubau', 'DUSCHE', 'Personenaufzug', 'ERSTBE...",wohnung,project,53124,6.0,8.2,9.6,"[{'label': 'Provisionsfrei', 'highlight': True}]",COMMISSION_FREE,Nürnberg,90429,Maximilianstraße 56,F2237DED191344059DB32E4F5C9709F0,53124,EcoLoft Gruppe AG,90429.0,Maximilianstraße,56,49.46008,11.04266,Seeleinsbühl,Nürnberg,483623.0,3.0,NaN,UNDEFINED,DE_ENV2014_CLASS_A_PLUS,24.5,NaN,3. Geschoss,NaN,NaN,Erstbezug,Neubau,NaN,Blockheizkraftwerk,KfW 55,Fußbodenheizung,Personenaufzug,NaN
4,WOHNUNG,X4_204,ZUM_KAUF,2023.0,75.52,493524,True,90429,EUR,"['Neubau', 'DUSCHE', 'Personenaufzug', 'Erdges...",wohnung,project,53124,6.0,8.2,9.6,"[{'label': 'Provisionsfrei', 'highlight': True}]",COMMISSION_FREE,Nürnberg,90429,Maximilianstraße 56,F2237DED191344059DB32E4F5C9709F0,53124,EcoLoft Gruppe AG,90429.0,Maximilianstraße,56,49.46008,11.04266,Seeleinsbühl,Nürnberg,433956.0,3.0,NaN,UNDEFINED,DE_ENV2014_CLASS_A_PLUS,24.5,NaN,2. Geschoss (Erdgeschoss),NaN,NaN,Erstbezug,Neubau,NaN,Blockheizkraftwerk,KfW 55,Fußbodenheizung,Personenaufzug,NaN


In [4]:
# data overview (with meta data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 961 entries, 0 to 960
Data columns (total 49 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   General.EstateTypeKey                                  961 non-null    object 
 1   General.ReferenceNumber                                832 non-null    object 
 2   General.DistributionTypeKey                            961 non-null    object 
 3   General.ConstructionYear                               961 non-null    float64
 4   General.LivingSpace                                    961 non-null    float64
 5   Tealium.object_locationid                              961 non-null    int64  
 6   Tealium.object_address_is_visible                      961 non-null    bool   
 7   Tealium.object_zip                                     961 non-null    int64  
 8   Tealium.object_currency                           

In [5]:
# show all variables in the data set
df.columns

Index(['General.EstateTypeKey', 'General.ReferenceNumber',
       'General.DistributionTypeKey', 'General.ConstructionYear',
       'General.LivingSpace', 'Tealium.object_locationid',
       'Tealium.object_address_is_visible', 'Tealium.object_zip',
       'Tealium.object_currency', 'Tealium.object_features',
       'Tealium.object_objektart_sub', 'Tealium.expose_type',
       'Tealium.broker_guid', 'Tealium.object_count_photos',
       'LocalRatings.scores.local_amenities', 'LocalRatings.scores.mobility',
       'MetaBadges', 'Price.AdditionalInformation.Commission.CommissionType',
       'EstateAddress.City', 'EstateAddress.ZipCode', 'EstateAddress.Street',
       'Offerer.addressGuid', 'Offerer.globalUserId',
       'Offerer.contactData.companyName', 'Project.Address.PostCode',
       'Project.Address.Street', 'Project.Address.HouseNumber', 'Latitude',
       'Longitude', 'Stadtteil', 'Stadt', 'PRICE', 'ROOMS',
       'PRICE_COMMONCHARGE', 'EnergyType', 'Class', 'Value', 'CATEGORY',

### Data corrections

In [6]:
#rename variables with preliminary categorization
df.rename(columns={
    # General Property Information
    'General.EstateTypeKey': 'estateTypeKey',
    'General.ReferenceNumber': 'referenceNumber',
    'General.DistributionTypeKey': 'distributionTypeKey',
    'General.ConstructionYear': 'constructionYear',
    'General.LivingSpace': 'livingSpace',

    # Tealium Object Information
    'Tealium.object_locationid': 'objectLocationId',
    'Tealium.object_address_is_visible': 'objectAddressIsVisible',
    'Tealium.object_zip': 'objectZip',
    'Tealium.object_currency': 'objectCurrency',
    'Tealium.object_features': 'objectFeatures',
    'Tealium.object_objektart_sub': 'objectObjektartSub',
    'Tealium.expose_type': 'exposeType',
    'Tealium.broker_guid': 'brokerGuid',
    'Tealium.object_count_photos': 'photosCount',

    # Local Ratings
    'LocalRatings.scores.local_amenities': 'localAmenitiesScore',
    'LocalRatings.scores.mobility': 'localMobilityScore',

    # Meta Information
    'MetaBadges': 'metaBadges',

    # Price and Commission Information
    'Price.AdditionalInformation.Commission.CommissionType': 'commissionType',
    'PRICE': 'price',
    'PRICE_COMMONCHARGE': 'priceCommoncharge',

    # Estate Address Information
    'EstateAddress.City': 'estateCity',
    'EstateAddress.ZipCode': 'estateZipCode',
    'EstateAddress.Street': 'estateStreet',

    # Offerer Information
    'Offerer.addressGuid': 'offererAddressGuid',
    'Offerer.globalUserId': 'offererGlobalUserId',
    'Offerer.contactData.companyName': 'offererCompanyName',

    # Project Address Information
    'Project.Address.PostCode': 'projectPostCode',
    'Project.Address.Street': 'projectStreet',
    'Project.Address.HouseNumber': 'projectHouseNumber',

    # Location Information
    'Latitude': 'latitudeDistrict',
    'Longitude': 'longitudeDistrict',
    'Stadtteil': 'district',
    'Stadt': 'town',

    # Energy Information
    'EnergyType': 'energyType',
    'Class': 'energyClass',
    'Value': 'energyConsumption',

    # Room Information
    'ROOMS': 'rooms',

    # Other Property Details
    'CATEGORY': 'category',
    'FLOOR': 'floor',
    'USAGE': 'usage',
    'SUITABILITY': 'suitability',
    'CONDITION': 'condition',
    'HOUSECONDITION': 'houseCondition',
    'PARKINGSLOT': 'parkingSlot',
    'ENERGY': 'energy',
    'BUILDINGTYPE': 'buildingType',
    'HEATING': 'heating',
    'ELEVATOR': 'elevator',
    'EQUIPMENTS': 'equipments'},
    inplace=True
    )

#### Duplikate

Wir suchen nach Duplikaten in den Referenznummern (referenceNumber) innerhalb desselben Maklers (Gruppierungsvariable: brokerGuid), da Duplikate nur innerhalb desselben Maklers auftreten können. 
Annahme: Eigentümer beauftragen nur einen Makler mit dem Verkauf, um mögliche höhere Maklerkosten zu vermeiden.

Zudem wird überprüft ob der Datensatz identische Datensätze enthält.

Variablen, die für diese Prüfungen verwendet wurden, werden im Anschluss aus dem DataFrame entfernt, wenn diese voraussichtlich keine weiteren Mehrwert bitten.

In [7]:
duplicateCount = df.groupby(['brokerGuid', 'referenceNumber']).size().reset_index(name='Duplicate Count').sort_values(by='Duplicate Count', ascending=False)
#print(duplicateCount)
if duplicateCount['Duplicate Count'].eq(1).all():
    print("Keine Duplikate innerhalb desselben Maklers gefunden")

# Überprüfung auf identische Zeilen
identicalRows = df[df.duplicated()]
if identicalRows.empty:
    print("Keine identischen Zeilen gefunden")

Keine Duplikate innerhalb desselben Maklers gefunden
Keine identischen Zeilen gefunden


In [8]:
df.drop(columns=['referenceNumber', 'brokerGuid'], inplace=True)

#### Einhaltung der ursprünglichen Suchkriterien

Für die fortlaufende Analyse sind wir an folgenden Kriterien interessiert:

- Nur Wohnungen werden berücksichtigt; andere Arten von Immobilien werden aus der Analyse ausgeschlossen.
- Wohnungen, die zwangsversteigert werden, werden nicht einbezogen, da ihre Verkaufspreise nicht marktnah sind.
- Es werden ausschließlich Wohnungen betrachtet, die zum Verkauf stehen.
- Beschränkung auf Wohnungen in der Stadt Nürnberg.

Immobilien, die diese Kriterien nicht erfüllen, werden aus dem Datensatz entfernt. Variablen, die für diese Prüfungen verwendet wurden, werden im Anschluss aus dem DataFrame entfernt, wenn diese voraussichtlich keine weiteren Mehrwert bieten.

In [9]:
# Count values
valueCountsList = [
    df['estateTypeKey'].value_counts().rename('estateTypeCount'),
    df['metaBadges'].value_counts().rename('metaBadgesCount'),
    df['distributionTypeKey'].value_counts().rename('distributionTypeKeyCount'),
    df['town'].value_counts().rename('townCount')
]

# Concatenate the value counts into one DataFrame with a multi-level index
combinedCounts = pd.concat(valueCountsList, axis=1, keys=[s.name for s in valueCountsList])

print(combinedCounts)

                                                    estateTypeCount  \
WOHNUNG                                                       959.0   
HAUS                                                            2.0   
[]                                                              NaN   
[{'label': 'Provisionsfrei', 'highlight': True}]                NaN   
[{'label': 'Privater Anbieter', 'highlight': Tr...              NaN   
[{'label': 'Zwangsversteigerung', 'highlight': ...              NaN   
ZUM_KAUF                                                        NaN   
Nürnberg                                                        NaN   

                                                    metaBadgesCount  \
WOHNUNG                                                         NaN   
HAUS                                                            NaN   
[]                                                            667.0   
[{'label': 'Provisionsfrei', 'highlight': True}]              253.0   
[{'la

In [10]:
# Löschen der Zeilen mit "estateTypeKey" "HAUS"
df.drop(df[df['estateTypeKey'].str.contains('HAUS')].index, inplace=True)
# Löschen Variablen
df = df.drop(columns=['estateTypeKey'])

In [11]:
# Löschen der Zeilen mit Label "Zwangsversteigerung" in der Spalte metaBadges
df = df[~df['metaBadges'].astype(str).str.contains("'label': 'Zwangsversteigerung'")]
# Löschen Variablen
df.drop(columns=['metaBadges'], inplace=True)

In [12]:
# Löschen Variablen
df.drop(columns=['distributionTypeKey'], inplace=True)

#### Geographische Variablen
Geographische Attribute wie Längengrad (latitudeDistrict) und Breitengrad (longitudeDistrict) des Stadtteils werden entfernt, da diese Informationen bereits in der Variable "Stadtteil" (district) enthalten sind. Die Distanz zwischen Stadtteilen wird nicht berücksichtigt. Da alle Immobilien in Nürnberg sind, wird die Variable "Stadt" (town) gelöscht. Da nur der Stadtteil von Interesse ist, werden folgende geographische Variablen entfernt:

- 'projectPostCode'
- 'projectStreet'
- 'projectHouseNumber'
- 'estateCity'
- 'estateZipCode'
- 'estateStreet'
- 'latitudeDistrict'
- 'longitudeDistrict'
- 'town'
- 'objectAddressIsVisible'
- 'objectAddressIsVisible'
- 'objectZip'

Die Löschung dieser geographischen Variablen dient der Datenbereinigung und ermöglicht eine effiziente und fokussierte Analyse auf die relevanten Informationen. 

In [13]:
columnsToDrop = [
    'projectPostCode', 'projectStreet', 'projectHouseNumber',
    'estateCity', 'estateZipCode', 'estateStreet',
    'latitudeDistrict', 'longitudeDistrict', 'town',
    'objectAddressIsVisible', 'objectZip', 'objectLocationId'
]

# Drop specified columns from the DataFrame
df.drop(columns=columnsToDrop, axis=1, inplace=True)

#### Anbieterinformationen
Die Informationen bezüglich des Anbieters der Immobilie sind nicht von Relevanz für unsere Analyse. Wir behalten ausschließlich die variable offererGlobalUserId, da diese Identifikationsnummer für jedes Objekt verfügbar ist und ausreichend ist, um den Anbieter zu unterscheiden. Daher werden alle anderen Informationen zum Anbieter entfernt, um die Datensätze zu bereinigen und die Analyse auf die relevanten Daten zu konzentrieren.

In [14]:
columnsToDrop = [
    'offererAddressGuid', 'offererCompanyName'
]

# Drop specified columns from the DataFrame
df.drop(columns=columnsToDrop, axis=1, inplace=True)

In [15]:
# show all variables in the data set
df.columns

Index(['constructionYear', 'livingSpace', 'objectCurrency', 'objectFeatures',
       'objectObjektartSub', 'exposeType', 'photosCount',
       'localAmenitiesScore', 'localMobilityScore', 'commissionType',
       'offererGlobalUserId', 'district', 'price', 'rooms',
       'priceCommoncharge', 'energyType', 'energyClass', 'energyConsumption',
       'category', 'floor', 'usage', 'suitability', 'condition',
       'houseCondition', 'parkingSlot', 'energy', 'buildingType', 'heating',
       'elevator', 'equipments'],
      dtype='object')

In [16]:
# show missing values
print(df.isnull().sum())

constructionYear         0
livingSpace              0
objectCurrency           0
objectFeatures           0
objectObjektartSub       0
exposeType               0
photosCount              1
localAmenitiesScore      3
localMobilityScore       3
commissionType           0
offererGlobalUserId      0
district                 0
price                    0
rooms                    0
priceCommoncharge      525
energyType             221
energyClass            326
energyConsumption      221
category               798
floor                  145
usage                  623
suitability            785
condition              533
houseCondition         731
parkingSlot            544
energy                 161
buildingType           830
heating                214
elevator               484
equipments             864
dtype: int64


# Next steps:
A Umgang mit fehlenden Werten

- Ableitung von energyClass aus energyConsumption, buildingType und energyType  --> Dies würde zu einer Reduzierung der fehlenden Werte führen. Anschließend könnten energyConsumption, buildingType und energyType gelöscht werden, da sie dieselben Informationen enthalten und wahrscheinlich eine hohe Korrelation aufweisen (dies kann auch zuvor überprüft werden).
- Ersatz des fehlenden Werts in photoCount durch "0", da in diesem Fall kein Bild übermittelt wurde.
- Ersatz der fehlenden Werte in den Variablen "elevator" und "parkingSlot" durch "0" mit der Begründung, dass dies als Verkaufsvorteil gilt und daher alle Immobilien, die über einen Aufzug oder Parkplatz verfügen, angegeben wurden.
- Imputation der fehlenden Werte in "LocalAmenities" und "MobilityScore" (finde ähnliche geografische Immobilien im Datensatz und übernehme den Wert).

!!!Um die Regression durchzuführen, dürfen keinerlei fehlende Werte vorhanden sein!!! Im Notfall müssen diese Zeilen gelöscht werden.

- Die folgenden Variablen können gelöscht werden, da zu viele fehlende Werte vorhanden sind: equipments, houseCondition, condition, suitability, usage, category, priceCommoncharge


B Erstellung neuer Variablen

- siehe weiter unten (Lena hat hier bereits sehr gute Arbeit geleistet)
- Stadtteile (distinct) in folgenden Gruppen unterteilen: Nord, Süd, West, Ost, Zentrum (oder eine andere Aufteilung - siehe: https://www.nuernberg.de/internet/statistik/stadtteile.html


C.1 # change data type
df['...'] = pd.Categorical(df['...'])
df['...'] = pd.Categorical(df['...'])
df['...'] = pd.Categorical(df['...'])

C Daten analysieren (in zwei Gruppen: categorial und nummerisch)
 - arithmetisch
 - visuell
 
 Korrelationsmatrix erstellen
 Outlier bestimmen (Begründen: Influencing?)
 
 D Bestimmung notwendiger Variablen um Model zu bauen

Fehlende Werte ersetzen

In [17]:
df.drop(columns=["equipments", "houseCondition", "condition", "suitability", "usage", "category", "priceCommoncharge"], inplace=True)

In [18]:
df["photosCount"].fillna(0, inplace=True)

In [19]:
df['localAmenitiesScore'] = df.groupby('district')['localAmenitiesScore'].transform(lambda x: x.fillna(x.mean()))
df['localMobilityScore'] = df.groupby('district')['localMobilityScore'].transform(lambda x: x.fillna(x.mean()))

In [20]:
df["elevator"] = df["elevator"].notnull().astype(int).fillna(0)

In [21]:
df["parkingSlot"] = df["parkingSlot"].notnull().astype(int).fillna(0)

In [22]:
equipments = {
    "möbliert": 1,
    "teilweise möbliert": 2, 
    "neuwertig": 3,
    "teilweise möbliert, neuwertig": 4,
    "möbliert, neuwertig": 5
}

In [23]:
energyClasses = {
    "DE_ENV2014_CLASS_A_PLUS": 1,
    "DE_ENV2014_CLASS_A": 2,
    "DE_ENV2014_CLASS_B": 3,
    "DE_ENV2014_CLASS_C": 4,
    "DE_ENV2014_CLASS_D": 5,
    "DE_ENV2014_CLASS_E": 6,
    "DE_ENV2014_CLASS_F": 7,
    "DE_ENV2014_CLASS_G": 8,
    "DE_ENV2014_CLASS_H": 9
}
df["energyClass"].replace(energyClasses, inplace=True)


In [24]:
df.isna().sum()

constructionYear         0
livingSpace              0
objectCurrency           0
objectFeatures           0
objectObjektartSub       0
exposeType               0
photosCount              0
localAmenitiesScore      0
localMobilityScore       0
commissionType           0
offererGlobalUserId      0
district                 0
price                    0
rooms                    0
energyType             221
energyClass            326
energyConsumption      221
floor                  145
parkingSlot              0
energy                 161
buildingType           830
heating                214
elevator                 0
dtype: int64

In [25]:
# Only include predictor & response variables
df_pred_resp = df[["constructionYear", "livingSpace",
    "photosCount", "localAmenitiesScore",
    "localMobilityScore", "rooms",
    "energyConsumption", "elevator",
    "price", "energyClass", "district"]
    ].copy()

In [26]:
df_pred_resp

,constructionYear,livingSpace,photosCount,localAmenitiesScore,localMobilityScore,rooms,energyConsumption,elevator,price,energyClass,district
0,2023.0,247.07,15.0,7.7,9.5,7.0,NaN,1,2539000.0,NaN,Mögeldorf
1,1900.0,69.00,23.0,6.5,9.7,2.0,NaN,0,225000.0,NaN,Gibitzenhof
2,1972.0,69.46,1.0,5.7,7.8,3.0,NaN,0,195000.0,NaN,Langwasser
3,2023.0,82.81,6.0,8.2,9.6,3.0,24.5,1,483623.0,1.0,Seeleinsbühl
4,2023.0,75.52,6.0,8.2,9.6,3.0,24.5,1,433956.0,1.0,Seeleinsbühl
...,...,...,...,...,...,...,...,...,...,...,...
956,2023.0,110.32,12.0,7.7,9.5,4.0,NaN,1,909000.0,NaN,Mögeldorf
957,2023.0,103.11,8.0,7.7,9.5,4.0,NaN,1,819000.0,NaN,Mögeldorf
958,1965.0,91.00,8.0,4.3,9.3,4.0,91.3,1,295000.0,NaN,Gibitzenhof
959,1958.0,34.00,18.0,8.9,9.7,1.0,118.3,0,198000.0,5.0,Maxfeld


### Variable lists

In [27]:
# Mit 100M+ Outlier
alt.Chart(df_pred_resp).mark_boxplot().encode(
    x=alt.X("price:Q").scale(zero=False)
    ).properties(width=1500, height=100)

alt.Chart(...)

In [28]:
# remove outlier for visualisation
df_pred_resp_vis = df_pred_resp.loc[df_pred_resp["price"] < 80000000]

In [29]:
# Ohne 100M+ Outlier
alt.Chart(df_pred_resp_vis).mark_boxplot().encode(
    x=alt.X("price:Q").scale(zero=False)
    ).properties(width=1500, height=100)

alt.Chart(...)

In [30]:
y_labels = ["price", "energyClass"]
x_labels = ["constructionYear", "livingSpace",
            "photosCount", "localAmenitiesScore",
            "localMobilityScore",
            "energyConsumption", "elevator"]

### Data splitting

## Analysis

### Descriptive statistics

In [31]:
df_pred_resp.describe().T

,count,mean,std,min,25%,50%,75%,max
constructionYear,948.0,1979.180380,3.654348e+01,1872.00,1957.00,1977.0,2019.25,2025.0
livingSpace,948.0,77.190264,3.556277e+01,19.35,56.95,73.0,90.00,532.0
photosCount,948.0,13.060127,9.490815e+00,0.00,7.00,11.0,16.00,54.0
localAmenitiesScore,948.0,6.714161,1.658366e+00,1.00,5.70,7.0,8.00,9.5
localMobilityScore,948.0,9.175994,9.197925e-01,1.50,9.10,9.6,9.70,9.7
rooms,948.0,2.757911,1.216392e+00,1.00,2.00,3.0,3.00,22.0
energyConsumption,727.0,102.342352,5.431241e+01,0.00,67.60,103.3,135.25,326.0
elevator,948.0,0.489451,5.001526e-01,0.00,0.00,0.0,1.00,1.0
price,948.0,463347.173523,3.244513e+06,74000.00,203750.00,295000.0,445175.00,100000000.0
energyClass,622.0,4.348875,1.785292e+00,1.00,3.00,5.0,6.00,9.0


### Exploratory data analysis

In [32]:
alt.renderers.enable('html')
alt.Chart(df_pred_resp_vis).mark_circle().encode(
    x=alt.X(alt.repeat("column"), 
            type='quantitative',
            scale=alt.Scale(zero=False)
            ),
    y=alt.Y(alt.repeat("row"), 
            type='quantitative',
            scale=alt.Scale(zero=False)
             )
).properties(
    width=150,
    height=150
).repeat(
    row=y_labels,
    column=x_labels
)

alt.RepeatChart(...)

### Relationships

## Model

### Select model

In [35]:
df_pred_resp[y_labels]

,price,energyClass
0,2539000.0,NaN
1,225000.0,NaN
2,195000.0,NaN
3,483623.0,1.0
4,433956.0,1.0
...,...,...
956,909000.0,NaN
957,819000.0,NaN
958,295000.0,NaN
959,198000.0,5.0


In [72]:
import statsmodels.api as sm

#X = sm.add_constant(df_pred_resp[["constructionYear", "livingSpace",  "localAmenitiesScore", "localMobilityScore", "elevator"]])
X = sm.add_constant(df_pred_resp[["constructionYear", "livingSpace", "elevator"]])
model = sm.OLS(df_pred_resp["price"], X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     17.91
Date:                Fri, 15 Dec 2023   Prob (F-statistic):           2.59e-11
Time:                        16:41:07   Log-Likelihood:                -15531.
No. Observations:                 948   AIC:                         3.107e+04
Df Residuals:                     944   BIC:                         3.109e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -6.243e+06   6.44e+06     -0.969      0.333   -1.89e+07    6.41e+06
constructionYear  2541.1899   3296.051      0.771      0.441   -3927.245    9009.625
livingSpace       2.032e+04   2904.205      6.996      0.000    1.46e+04     2.6e+04
elevator           2.22e+05    2.4e+05      0.926      0.355   -2.49e+05    6.93e+05
==============================================================================
Omnibus:                     2525.222   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         31017907.591
Skew:                          29.272   Prob(JB):                         0.00
Kurtosis:                     887.215   Cond. No.                     1.24e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.24e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [73]:
df_pred_resp["predicted"] = model.predict()

In [76]:
data = ((df_pred_resp["predicted"] - df_pred_resp["price"])/df_pred_resp["price"])*100
#data = df_pred_resp["predicted"] - df_pred_resp["price"]
#sns.scatterplot(data=data)
import plotly.express as px
px.scatter(data)

In [41]:
X.isna().sum()

const                    0
constructionYear         0
livingSpace              0
photosCount              0
localAmenitiesScore      0
localMobilityScore       0
energyConsumption      221
elevator                 0
dtype: int64

### Training and validation

### Fit model

### Evaluation on test set

### Save model



Save your model in the folder `models/`. Use a meaningful name and a timestamp.

## Conclusions